In [1]:
import sys
sys.path.append( '../' )

In [2]:
import numpy as np
import cs_vqe as c
import ast
import os
import scipy as sp
from copy import deepcopy

In [3]:
from openfermion import qubit_operator_sparse
import quchem.Misc_functions.conversion_scripts as conv_scr
from openfermion.ops import QubitOperator

In [4]:
# with open("hamiltonians.txt", 'r') as input_file:
#     hamiltonians = ast.literal_eval(input_file.read())
    
cwd = os.getcwd()
working_dir = os.path.dirname(cwd)
data_dir = os.path.join(working_dir, 'data')
data_hamiltonians_file = os.path.join(data_dir, 'hamiltonians.txt')


with open(data_hamiltonians_file, 'r') as input_file:
    hamiltonians = ast.literal_eval(input_file.read())

In [5]:
for key in hamiltonians.keys():
    print(f"{key: <25}     n_qubits:  {hamiltonians[key][1]:<5.0f}")

H2-S1_STO-3G_singlet          n_qubits:  18   
C1-O1_STO-3G_singlet          n_qubits:  16   
H1-Cl1_STO-3G_singlet         n_qubits:  16   
H1-Na1_STO-3G_singlet         n_qubits:  16   
H2-Mg1_STO-3G_singlet         n_qubits:  17   
H1-F1_3-21G_singlet           n_qubits:  18   
H1-Li1_3-21G_singlet          n_qubits:  18   
Be1_STO-3G_singlet            n_qubits:  5    
H1-F1_STO-3G_singlet          n_qubits:  8    
H1-Li1_STO-3G_singlet         n_qubits:  8    
Ar1_STO-3G_singlet            n_qubits:  13   
F2_STO-3G_singlet             n_qubits:  15   
H1-O1_STO-3G_singlet          n_qubits:  8    
H2-Be1_STO-3G_singlet         n_qubits:  9    
H2-O1_STO-3G_singlet          n_qubits:  10   
H2_3-21G_singlet              n_qubits:  5    
H2_6-31G_singlet              n_qubits:  5    
H3-N1_STO-3G_singlet          n_qubits:  13   
H4-C1_STO-3G_singlet          n_qubits:  14   
Mg1_STO-3G_singlet            n_qubits:  13   
N2_STO-3G_singlet             n_qubits:  15   
Ne1_STO-3G_si

In [6]:
data_csvqe_results_file = os.path.join(data_dir, 'csvqe_results.txt')
with open(data_csvqe_results_file, 'r') as input_file:
    csvqe_results = ast.literal_eval(input_file.read())

for key in csvqe_results.keys():
    print(f"{key: <25}     n_qubits:  {hamiltonians[key][1]:<5.0f}")

H3_STO-3G_singlet_1+          n_qubits:  3    
Be1_STO-3G_singlet            n_qubits:  5    
H2_3-21G_singlet              n_qubits:  5    
H2_6-31G_singlet              n_qubits:  5    
H1-He1_3-21G_singlet_1+       n_qubits:  6    
H1-F1_STO-3G_singlet          n_qubits:  8    
H1-Li1_STO-3G_singlet         n_qubits:  8    
H1-O1_STO-3G_singlet          n_qubits:  8    
H2-Be1_STO-3G_singlet         n_qubits:  9    
H3_3-21G_singlet_1+           n_qubits:  9    
H2-O1_STO-3G_singlet          n_qubits:  10   
H3-N1_STO-3G_singlet          n_qubits:  13   
Mg1_STO-3G_singlet            n_qubits:  13   
H4-C1_STO-3G_singlet          n_qubits:  14   
H4-N1_STO-3G_singlet_1+       n_qubits:  14   
F2_STO-3G_singlet             n_qubits:  15   
N2_STO-3G_singlet             n_qubits:  15   
O2_STO-3G_singlet             n_qubits:  15   
C1-O1_STO-3G_singlet          n_qubits:  16   
H1-Cl1_STO-3G_singlet         n_qubits:  16   
H1-Na1_STO-3G_singlet         n_qubits:  16   
H2-Mg1_STO-3G

In [7]:
# mol_k
# mol_key ='H3_STO-3G_singlet_1+'
# mol_key ='H2-Be1_STO-3G_singlet'
mol_key = 'H1-Li1_STO-3G_singlet'


In [8]:
print(f"n_qubits:  {hamiltonians[mol_key][1]}")

n_qubits:  8


# Get non-contextual H

In [9]:
ham = hamiltonians[mol_key][2]
ham

{'IIIIIIII': -5.001425458221718,
 'IIIIIIIX': -0.0005602367691690761,
 'IIIIIIIZ': 1.0104418175624195,
 'IIIIIIXI': 0.01115672755331298,
 'IIIIIIXZ': 0.0002823107691970577,
 'IIIIIIYY': -0.0018494520120760328,
 'IIIIIIZI': 1.0104418175624192,
 'IIIIIIZZ': 0.41455546243330654,
 'IIIIIXIX': -0.02906042590649248,
 'IIIIIXXI': 9.308532583860961e-05,
 'IIIIIXYY': -0.0020610093726644778,
 'IIIIIXZX': -0.015075372035589736,
 'IIIIIXZZ': -0.03495312426599749,
 'IIIIIYIY': -0.02906042590649248,
 'IIIIIYYX': 0.0020610093726644778,
 'IIIIIYZY': -0.015075372035589736,
 'IIIIIZII': -0.11536854093063532,
 'IIIIIZIZ': 0.0908717615562679,
 'IIIIIZXZ': -0.0015283726900879773,
 'IIIIIZZI': 0.09450446372057475,
 'IIIIXIXI': 0.001783048244458605,
 'IIIIXIZZ': -0.03290395741989794,
 'IIIIXXIX': 0.001407793599187085,
 'IIIIXXYY': -0.0036327021643068054,
 'IIIIXYIY': 0.001407793599187085,
 'IIIIXYYX': 0.0036327021643068054,
 'IIIIXZIZ': 0.005477331994841993,
 'IIIIXZXI': -0.015075372035589731,
 'IIIIXZXZ': -

In [10]:
n_qubits = hamiltonians[mol_key][1]

ham_noncon = hamiltonians[mol_key][3]
Con_H = {}

for P in ham:
    if P in ham_noncon:
        continue
    else:
        Con_H[P]=ham[P]
        

## Testing contextuality

In [11]:
print('Is NONcontextual correct:', not c.contextualQ_ham(ham_noncon))
print('Is contextual correct:',c.contextualQ_ham(Con_H))

Is NONcontextual correct: True
Is contextual correct: True


# Classical part of problem!

Take $H_{nc}$ and split into:
- $Z$ = operators that completely comute with all operators in $S$
- $T$ = remaining operators in $S$
    - where $S = Z \cup T$  and $S$ is set of Pauli operators in $H_{nc}$
    
    
- We then split the set $T$ into cliques $C_{1}, C_{2}, ... , C_{|T|}$
    - all ops in a clique commute
    - ops between cliques anti-commute!

In [12]:
model = c.quasi_model(ham_noncon)
fn_form = c.energy_function_form(ham_noncon, model)

# Function form

$$R = G \cup \{ C_{i1} \;| \; i=1,2,...,N \}$$

- note q to do with $G$
- note r to do with $C_{i1}$

In [13]:
Energy_function = c.energy_function(fn_form)

In [14]:
ground_state_params = [list(hamiltonians[mol_key][5][1]), list(hamiltonians[mol_key][5][2])]
ground_state_params

[[1, 1, 1, 1, 1, -1, -1], [-1.0575336511480974e-07, 0.9999999999999944]]

In [15]:
## check

H_symbolic = conv_scr.Get_Openfermion_Hamiltonian(ham_noncon)
sparseH = qubit_operator_sparse(H_symbolic, n_qubits=n_qubits)

if sparseH.shape[0]<128:
    eig_vals, eig_vecs = sp.linalg.eigh(sparseH.todense())
    lowest_eigenvalue = min(eig_vals)
else:
    lowest_eigenvalue = sp.sparse.linalg.eigsh(sparseH, which='SA', k=1)[0][0]
    
np.isclose(Energy_function(*ground_state_params[0],*ground_state_params[1]),
           lowest_eigenvalue)

True

# Now need to rotate Hamiltonian!

We now have non contextual ground state: $(\vec{q}, \vec{r})$

In [16]:
ground_state_params

[[1, 1, 1, 1, 1, -1, -1], [-1.0575336511480974e-07, 0.9999999999999944]]

We can use this result - ground state of $H_{nc}$ -  as a classical estiamte of our ground state of the full Hamiltonian ($H = H_{c} + H_{nc}$)

However we can also obtain a quantum correction using $H_{c}$

By minimizing theenergy of the remaining terms in the Hamiltonian over the quantum states that are **consistent with the noncon-textual ground state**.

To do this we first rotate each $G_{j}$ and $\mathcal{A} = \sum_{i=1}^{N} r_{i}A_{i}$:

In [17]:
true_gs = hamiltonians[mol_key][4]
true_gs

-8.933276065676191

In [18]:
order = csvqe_results[mol_key][-1] # <-- order specified from greedy run by Will
qubit_removal_order=deepcopy(order)
qubit_removal_order

[4, 7, 2, 3, 0, 1, 5, 6]

# Rotate full Hamiltonian to basis with diagonal noncontextual generators!

In [19]:
from cs_vqe_updated_up import diagonalize_epistemic_dictionary_generator

In [20]:
(mapping_GuA_to_singleZ_with_ep_exp_vals, 
 R_op_dict,
GuA_full_rotated) = diagonalize_epistemic_dictionary_generator(model, 
                                                            fn_form, 
                                                            ground_state_params, 
                                                            check_reduction=False,
                                                             up_method='LCU')
R_op_dict

{'unitary_part_method': 'LCU',
 'R_op': 0.7071067437970858 [] +
 -0.7071068185760073j [Z0 Z1 Z3 Y5 Z6]}

In [21]:
from cs_vqe_updated_up import get_wills_order

qubit_removal_order=deepcopy(order)

actual_qubit_removal_order = get_wills_order(GuA_full_rotated, qubit_removal_order)
actual_qubit_removal_order

[4, 5, 2, 3, 0, 1, 6, 7]

In [22]:
mapping_GuA_to_singleZ_with_ep_exp_vals

{'ZIIIIIII': {'single_Z': 'ZIIIIIII',
  'noncon_gs_exp_val': 1,
  'do_unitary_part': False},
 'IZIIIIII': {'single_Z': 'IZIIIIII',
  'noncon_gs_exp_val': 1,
  'do_unitary_part': False},
 'IIZIZIZI': {'single_Z': 'IIZIIIII',
  'noncon_gs_exp_val': 1,
  'do_unitary_part': False},
 'IIIZIIII': {'single_Z': 'IIIZIIII',
  'noncon_gs_exp_val': 1,
  'do_unitary_part': False},
 'IIIIZIZI': {'single_Z': 'IIIIZIII',
  'noncon_gs_exp_val': 1,
  'do_unitary_part': False},
 'IIIIIIZI': {'single_Z': 'IIIIIIZI',
  'noncon_gs_exp_val': -1,
  'do_unitary_part': False},
 'IIIIIIIZ': {'single_Z': 'IIIIIIIZ',
  'noncon_gs_exp_val': -1,
  'do_unitary_part': False},
 'IIIIIXZZ': {'single_Z': 'IIIIIZII',
  'noncon_gs_exp_val': 1,
  'do_unitary_part': True}}

In [23]:
from cs_vqe_updated_up import diagonalize_epistemic_by_fixed_qubit

In [24]:
(rotations, 
 do_unitary_part, 
 ep_state_trans,
 GuA) = diagonalize_epistemic_by_fixed_qubit(model, mapping_GuA_to_singleZ_with_ep_exp_vals, [2])
GuA

['IIZIIIII']

In [25]:
from cs_vqe_updated_up import get_reduced_hamiltonian_by_qubits_fixed

In [26]:
H_reduced = get_reduced_hamiltonian_by_qubits_fixed(ham,
                                             model, 
                                             mapping_GuA_to_singleZ_with_ep_exp_vals,
                                             [0],
                                             R_op_dict)
# H_reduced

In [27]:
from cs_vqe_updated_up import get_reduced_hamiltonians_by_order 


H_reduced_list = get_reduced_hamiltonians_by_order(ham,
                                             model, 
                                             mapping_GuA_to_singleZ_with_ep_exp_vals,
                                             actual_qubit_removal_order, # <-- actual GOOD order!
                                             R_op_dict)
H_reduced_list

[{'': (-8.914131221774564+0j)},
 {'I': (-8.83535229684981+0j),
  'Z': (0.07877892492475086+0j),
  'X': (-2.5476657034007033e-09+0j)},
 {'II': (-8.55716788750159+0j),
  'IX': (-0.07071961726976403+0j),
  'IZ': (0.27818441541848926+0j),
  'ZI': (0.09182499402100118+0j),
  'ZZ': (0.013046069096250341+0j),
  'ZX': (-0.07071961249958325+0j),
  'XI': (-0.0024088994356650803+0j),
  'XX': (0.03360551388322442+0j),
  'XZ': (-0.0024088968879994004+0j)},
 {'III': -8.247664329411942,
  'IIX': -0.06990624853199498,
  'IIZ': 0.2779730772187568,
  'IXI': 0.0020491668460995496,
  'IXZ': 0.0020491644610091484,
  'IYY': -0.06152341066669575,
  'IZI': -0.035451497763494405,
  'IZZ': 0.009125052203652603,
  'IZX': -0.00491169765978737,
  'XII': 0.0018392514814537762,
  'XXI': 0.0012044497178325402,
  'XXX': -0.01680275694161221,
  'XZZ': 0.010523510254282085,
  'XIZ': 0.010523510254282085,
  'XXZ': 0.0012044484439997002,
  'XZI': 0.0018392514814537762,
  'YIY': 0.015570388982752396,
  'YYI': 0.00120444971

In [28]:
qubit_removal_order=deepcopy(order)
qubit_removal_order
print(qubit_removal_order)
print()

red_Hs = c.get_reduced_hamiltonians(ham,
                           model,
                           fn_form,
                           ground_state_params,
                           qubit_removal_order)
red_Hs

[4, 7, 2, 3, 0, 1, 5, 6]



[{'': -8.914131227844832},
 {'I': -8.835352295441243,
  'Z': 0.07877893240358791,
  'X': -6.10156187206153e-09},
 {'II': -8.55716788750159,
  'IZ': -0.2781844079396502,
  'IX': 0.07071964668870159,
  'ZI': 0.09182499402100118,
  'ZX': 0.07071961387924856,
  'ZZ': -0.013046061617413285,
  'XI': -0.0024088994356650803,
  'XZ': 0.0024088933341032156,
  'XX': -0.033605514137973184},
 {'III': -8.247664329411942,
  'IIZ': -0.2779730698259343,
  'IIX': 0.06990627792858284,
  'YYI': 0.0020491668460995496,
  'YYX': 2.1670603697038686e-10,
  'YYZ': -0.002049164461009139,
  'YXY': 0.06152341066669575,
  'IZI': -0.035451497763494405,
  'IZX': 0.0049116986247923185,
  'IZZ': -0.009125051684224006,
  'XZI': 0.0018392514814537762,
  'ZXI': 0.0012044497178325402,
  'ZXZ': -0.0012044466670516076,
  'ZXX': 0.016802757068986596,
  'XIX': 1.1128966198738487e-09,
  'XIZ': -0.010523510254282026,
  'XZX': 1.1128966198738487e-09,
  'XZZ': -0.010523510254282026,
  'XII': 0.0018392514814537762,
  'YIY': 0.01557

In [29]:
len(H_reduced_list) == len(red_Hs)

True

In [30]:
E_list_one =[]
for H in red_Hs:
    H = {P_key: coeff.real for P_key, coeff in H.items() if not np.isclose(coeff.real,0)}
    
    H_mat = conv_scr.Get_Openfermion_Hamiltonian(H)
    sparseH = qubit_operator_sparse(H_mat, n_qubits=hamiltonians[mol_key][1])
    E = sp.sparse.linalg.eigsh(sparseH, which='SA', k=1)[0][0]
#     print(E)
#     print(len(H))
#     print()
    E_list_one.append(E)

In [31]:
E_list_two =[]
for H in H_reduced_list:
    H = {P_key: coeff.real for P_key, coeff in H.items() if not np.isclose(coeff.real,0)}
    H_mat = conv_scr.Get_Openfermion_Hamiltonian(H)
    sparseH = qubit_operator_sparse(H_mat, n_qubits=hamiltonians[mol_key][1])
    E = sp.sparse.linalg.eigsh(sparseH, which='SA', k=1)[0][0]
#     print(E)
#     print(len(H))
#     print()
    E_list_two.append(E)

In [32]:
np.array(E_list_two) - np.array(E_list_one)

array([ 6.07026784e-09,  6.07027317e-09, -3.55271368e-15,  1.06957998e-10,
        2.72763145e-10, -7.77356490e-09, -7.95191113e-09, -6.40166498e-09,
       -5.28476285e-09])

In [33]:
# cs_VQE new
np.array(E_list_two) - true_gs

array([ 1.91448439e-02,  1.91448439e-02,  1.73089963e-02,  3.68949038e-03,
        1.33588858e-03,  7.70803367e-04,  2.22706909e-04,  1.70409960e-04,
       -8.88178420e-15])

In [34]:
# cs_VQE will
np.array(E_list_one) - true_gs

array([1.91448378e-02, 1.91448378e-02, 1.73089963e-02, 3.68949027e-03,
       1.33588831e-03, 7.70811141e-04, 2.22714861e-04, 1.70416362e-04,
       5.28475397e-09])

In [35]:
# look for: do_unitary_part=TRUE
mapping_GuA_to_singleZ_with_ep_exp_vals


{'ZIIIIIII': {'single_Z': 'ZIIIIIII',
  'noncon_gs_exp_val': 1,
  'do_unitary_part': False},
 'IZIIIIII': {'single_Z': 'IZIIIIII',
  'noncon_gs_exp_val': 1,
  'do_unitary_part': False},
 'IIZIZIZI': {'single_Z': 'IIZIIIII',
  'noncon_gs_exp_val': 1,
  'do_unitary_part': False},
 'IIIZIIII': {'single_Z': 'IIIZIIII',
  'noncon_gs_exp_val': 1,
  'do_unitary_part': False},
 'IIIIZIZI': {'single_Z': 'IIIIZIII',
  'noncon_gs_exp_val': 1,
  'do_unitary_part': False},
 'IIIIIIZI': {'single_Z': 'IIIIIIZI',
  'noncon_gs_exp_val': -1,
  'do_unitary_part': False},
 'IIIIIIIZ': {'single_Z': 'IIIIIIIZ',
  'noncon_gs_exp_val': -1,
  'do_unitary_part': False},
 'IIIIIXZZ': {'single_Z': 'IIIIIZII',
  'noncon_gs_exp_val': 1,
  'do_unitary_part': True}}

In [36]:
for Z_key in mapping_GuA_to_singleZ_with_ep_exp_vals:
    if mapping_GuA_to_singleZ_with_ep_exp_vals[Z_key]['do_unitary_part'] is True:
        print('qubit ind where unitary part is done:',Z_key.index('Z'))

qubit ind where unitary part is done: 6


In [37]:
actual_qubit_removal_order

[4, 5, 2, 3, 0, 1, 6, 7]

In [38]:
order

[4, 7, 2, 3, 0, 1, 5, 6]

In [39]:
order

[4, 7, 2, 3, 0, 1, 5, 6]

In [40]:
from cs_vqe_updated_up import get_reduced_H_minimal_rotations

In [41]:
order = deepcopy(csvqe_results[mol_key][-1]) # <-- order specified from greedy run by Will

H_reduced_seqrot_csvqe = get_reduced_H_minimal_rotations(ham,
                                             model, 
                                             fn_form,
                                             ground_state_params,
                                             order,
                                             check_reduction=True,
                                             up_method='SeqRot')


In [42]:
order = deepcopy(csvqe_results[mol_key][-1]) # <-- order specified from greedy run by Will

H_reduced_seqrot_LCU = get_reduced_H_minimal_rotations(ham,
                                             model, 
                                             fn_form,
                                             ground_state_params,
                                             order,
                                             check_reduction=True,
                                             up_method='LCU')

In [43]:
for i in range(0, len(H_reduced_seqrot_LCU)):
    print(len(H_reduced_seqrot_csvqe['H_csvqe'][i]))
    print(len(H_reduced_seqrot_LCU['H_csvqe'][i]))
    print()

1
1

3
3

9
9

30
30



In [44]:
for ind, H_std in enumerate(H_reduced_seqrot_LCU['H_csvqe']):
    print(len(H_std))

1
3
9
30
100
155
231
366
558


In [45]:
hamiltonians[mol_key][4]

-8.933276065676191

In [46]:
csvqe_results[mol_key][2]

[0.01914483783133214,
 0.019144837831333916,
 0.017308996341794725,
 0.003689490268222073,
 0.0013358883064213956,
 0.0007708111406685703,
 0.00022271486088598635,
 0.00017041636165515683,
 5.2847344278461605e-09]

In [49]:
csvqe_results[mol_key][1]

[-53.248970562330506,
 -53.248970562330534,
 -53.25754457502269,
 -53.257611254192035,
 -53.266269409693344,
 -53.268651926948884,
 -53.27376966756567,
 -53.28056799105157,
 -53.288184741834975]

In [ ]:
mol_num = 2
molecules_and_qubits = [(mol_key, hamiltonians[mol_key][1])for mol_key in csvqe_results]
sorted_by_qubit_No = sorted(molecules_and_qubits, key= lambda x: x[1])
mol_key = sorted_by_qubit_No[mol_num-1][0] # UCL supercomputer indexes from 1, hence minus one here!
hamiltonians[mol_key][1]